In [39]:
!pip install requests
!pip install psycopg2
!pip install pandas

In [40]:
!pip install scikit-learn

In [41]:
import requests
import psycopg2
from psycopg2 import sql
import json
import math
import itertools

url_api = 'https://arqui-sistema-recomendacion-85b7038cdf33.herokuapp.com/api/inmueblesPorUsuario/get_filtro/'
url_api_inmuebles = 'https://arqui-sistema-recomendacion-85b7038cdf33.herokuapp.com/api/inmuebles/'
url_api_login = 'https://arqui-sistema-recomendacion-85b7038cdf33.herokuapp.com/api/login/'
url_api_preferenciasPorUsuario = 'https://arqui-sistema-recomendacion-85b7038cdf33.herokuapp.com/api/interesesPorUsuario/'

login_data = {
    'username': 'diego3026',
    'password': '3174748557d'
}

In [42]:
def normalizacion(valor,valor_maximo,valor_minimo):
    valor_normalizado = (valor - valor_minimo) / (valor_maximo - valor_minimo) 
    return valor_normalizado

In [43]:
class InmuebleUsuarioNormalizado():

    def __init__(self,inmueble,usuario,favorito,calificacion,clics,puntajePreferencia,pesos):
        self.inmueble = inmueble
        self.usuario = usuario
        self.favorito = favorito
        self.calificacion = calificacion
        self.clics = clics
        self.pesos = pesos
        self.puntajePreferencia = puntajePreferencia

    def get_clasificacion(self):
        clasificacion = self.pesos[0] * self.favorito + self.pesos[1] * self.calificacion + self.pesos[2] * self.clics + self.pesos[3] * self.puntajePreferencia
        return clasificacion


In [44]:
def get_datos_api():
    response_login = requests.post(url=url_api_login, data=login_data)

    if response_login.status_code == 200:
        data = response_login.json()
    else:
        print(f'Error en la petición login: {response_login.status_code}')


    token = data['access']
    headers = {'Authorization': f'Bearer {token}'}

    response = requests.get(url_api, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return 'Error en la petición: {response.status_code}'

In [45]:
def get_datos_preferenciasPorUsuario():
    response_login = requests.post(url=url_api_login, data=login_data)
    
    if response_login.status_code == 200:
        data = response_login.json()
    else:
        print(f'Error en la petición login: {response_login.status_code}')
        return None
    token = data['access']
    headers = {'Authorization': f'Bearer {token}'}

    response_preferenciasPorUsuario = requests.get(url_api_preferenciasPorUsuario, headers=headers)
    
    if response_preferenciasPorUsuario.status_code == 200:
        data_preferenciasPorUsuario = response_preferenciasPorUsuario.json()
        print(data_preferenciasPorUsuario)
        return data_preferenciasPorUsuario
    else :
        print(f'Error en la peticion de preferenciaPorUsuario: {response_preferenciasPorUsuario}')
        return None
        

In [46]:
preferencias = get_datos_preferenciasPorUsuario()

[{'id': 1, 'interes': 'piscina', 'usuario': 'diego3026'}, {'id': 2, 'interes': 'wifi', 'usuario': 'diego3026'}, {'id': 3, 'interes': 'cocina', 'usuario': 'diego3026'}, {'id': 4, 'interes': 'parque', 'usuario': 'diego3026'}, {'id': 5, 'interes': 'gimnasio', 'usuario': 'diego1234'}, {'id': 6, 'interes': 'estacionamiento', 'usuario': 'diego1234'}, {'id': 7, 'interes': 'vista-al-mar', 'usuario': 'diego1234'}, {'id': 8, 'interes': 'terraza', 'usuario': 'diego1234'}, {'id': 9, 'interes': 'cerca-de-transporte-publico', 'usuario': 'diego1234'}]


In [47]:
intereses_por_usuario = {}
for preferencia in preferencias:
    usuario = preferencia['usuario']
    interes = preferencia['interes']
    if usuario not in intereses_por_usuario:
        intereses_por_usuario[usuario] = []
    intereses_por_usuario[usuario].append(interes) 
print(intereses_por_usuario)

{'diego3026': ['piscina', 'wifi', 'cocina', 'parque'], 'diego1234': ['gimnasio', 'estacionamiento', 'vista-al-mar', 'terraza', 'cerca-de-transporte-publico']}


In [48]:
def consultar_base_de_datos(host, database, user, password, consulta, parametros=None):
    """
    Conecta a la base de datos PostgreSQL y ejecuta una consulta.

    :param host: Dirección del servidor PostgreSQL.
    :param database: Nombre de la base de datos.
    :param user: Nombre de usuario para la conexión.
    :param password: Contraseña del usuario.
    :param consulta: Consulta SQL a ejecutar.
    :param parametros: Parámetros para la consulta SQL (opcional).
    :return: Resultados de la consulta.
    """
    try:
        # Establecer la conexión a la base de datos
        conexion = psycopg2.connect(
            host=host,
            database=database,
            user=user,
            password=password
        )
        cursor = conexion.cursor()

        # Ejecutar la consulta
        cursor.execute(consulta)

        nombres_columnas = [desc[0] for desc in cursor.description]

        # Obtener los resultados de la consulta
        resultados = cursor.fetchall()

        # Cerrar el cursor y la conexión
        cursor.close()
        conexion.close()

        return nombres_columnas, resultados
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(f'Error al acceder a la base de datos: {error}')
        return None

In [49]:
def get_datos_api_inmuebles():
    response_login = requests.post(url=url_api_login, data=login_data)

    if response_login.status_code == 200:
        data = response_login.json()
    else:
        return f'Error en la petición login: {response_login.status_code}'


    token = data['access']
    headers = {'Authorization': f'Bearer {token}'}

    response = requests.get(url_api_inmuebles, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return 'Error en la petición: {response.status_code}'

In [50]:
host = 'dbarquitecura.postgres.database.azure.com'
database = 'postgres'
user = 'gidsyc'
password = 'Semillero2024'
consulta = 'SELECT id FROM inmobiliaria_inmueble'
consulta_user = 'SELECT id, username FROM inmobiliaria_usuario'

columnas, resultados = consultar_base_de_datos(host, database, user, password, consulta)

columnas_user, resultados_user = consultar_base_de_datos(host, database, user, password, consulta_user)

datosLimpios = {}

# if resultados:
#     print(columnas)
#     for fila in resultados:
#         print(fila[0])
# else:
#     print('No se obtuvieron resultados.')


# print("Usuarios")
# if resultados_user:
#     print(columnas_user)
#     for fila in resultados_user:
#         print(fila[1])
# else:
#     print('No se obtuvieron resultados.')


for item in resultados_user:
    usuario = item[1]
    if usuario not in datosLimpios:
        datosLimpios[usuario] = {}
        for inmueble in resultados:
            id_inmueble = inmueble[0]
            datosLimpios[usuario][id_inmueble] = 0
    else:
        print(f"El inmueble '{id_inmueble}' ya existe")


In [51]:
inmuebles = get_datos_api_inmuebles()

In [52]:
inmueble_caracteristicas = []
for item in inmuebles:
    caracteristicas = item['caracteristicas']
    if (len(caracteristicas) > 0):
        for caracteristica in caracteristicas:
            nombre = caracteristica['nombre'].lower()
            dato = {'id_inmueble': item['id'], 'nombre_caracteristica': nombre}
            inmueble_caracteristicas.append(dato)


In [53]:
inmuebles_con_caracteristicas = {}
for key, group in itertools.groupby(sorted(inmueble_caracteristicas, key=lambda x: x['id_inmueble']), lambda x: x['id_inmueble']):
    inmuebles_con_caracteristicas[key] = [item['nombre_caracteristica'] for item in group]
print(inmuebles_con_caracteristicas)


{1173: ['piso-13'], 1174: ['citofonos', 'estudio-o-biblioteca', 'ascensor', 'piscina', 'vista-panoramica', 'conjunto-cerrado', 'hall-de-alcobas', 'zona-de-lavanderia', 'salon-comunal', 'zonas-verdes', 'bano-auxiliar', 'cocina-integral', 'banos-de-servicio', 'terraza-balcon-balcon', 'vigilancia', 'con-terraza', 'porteria', 'aire-acondicionado', 'tipo-acabado-de-piso-ceramica', 'piso-11'], 1175: ['tipo-de-casa-tradicional'], 1176: ['zona-de-lavanderia', 'piso-7'], 1177: ['ascensor', 'vigilancia-24hrs', 'instalacion-de-gas-natural', 'calentador-caldera', 'tipo-de-comedor-sala-comedor', 'vigilancia', 'tipo-acabado-de-piso-ceramica', 'tipo-de-piso-en-alcobas-ceramica', 'tipo-de-piso-en-comedor-ceramica', 'tipo-de-piso-en-estudio-ceramica', 'tipo-de-piso-en-sala-ceramica', 'tipo-de-estufa-gas'], 1178: ['hall-de-alcobas', 'zona-para-ninos', 'bano-auxiliar', 'banos-de-servicio', 'piso-2', 'tipo-acabado-de-piso-ceramica', 'area-lote-357-m2'], 1179: ['citofonos', 'ascensor', 'piscina', 'conjunto

In [54]:
coincidencias_todos_usuarios = {}
for usuario, intereses in intereses_por_usuario.items():
    coincidencias_usuario = []
    for inmueble, caracteristicas in inmuebles_con_caracteristicas.items():
        coincidencias_usuario_inmueble = [interes for interes in intereses if interes in caracteristicas]
        if coincidencias_usuario_inmueble:
            coincidencias_usuario.append((inmueble, coincidencias_usuario_inmueble))
    coincidencias_todos_usuarios[usuario] = coincidencias_usuario

for usuario, coincidencias in coincidencias_todos_usuarios.items():
    print(f"Coincidencias para {usuario}:")
    for coincidencia in coincidencias:
        print(f"Inmueble {coincidencia[0]}: {coincidencia[1]}")
    print()

Coincidencias para diego3026:
Inmueble 1174: ['piscina']
Inmueble 1179: ['piscina']
Inmueble 1181: ['piscina']
Inmueble 1182: ['piscina']
Inmueble 1185: ['piscina']
Inmueble 1188: ['piscina']
Inmueble 1189: ['piscina']
Inmueble 1193: ['piscina']
Inmueble 1194: ['piscina']
Inmueble 1197: ['piscina']
Inmueble 1202: ['piscina']
Inmueble 1203: ['piscina']
Inmueble 1204: ['piscina']
Inmueble 1210: ['piscina']
Inmueble 1212: ['piscina']
Inmueble 1215: ['piscina']
Inmueble 1217: ['piscina']
Inmueble 1218: ['piscina']
Inmueble 1222: ['piscina']
Inmueble 1226: ['piscina']
Inmueble 1228: ['piscina']
Inmueble 1231: ['piscina']
Inmueble 1232: ['piscina']
Inmueble 1235: ['piscina']
Inmueble 1237: ['piscina']
Inmueble 1239: ['piscina']
Inmueble 1240: ['piscina']
Inmueble 1246: ['piscina']
Inmueble 1248: ['piscina']
Inmueble 1249: ['piscina']
Inmueble 1251: ['piscina']
Inmueble 1253: ['piscina']
Inmueble 1255: ['piscina']
Inmueble 1257: ['piscina']
Inmueble 1262: ['piscina']
Inmueble 1263: ['piscina'

In [55]:
def valoresCoincidenciaPorUsuario(maxPuntaje: int):
    valoresPorUsuarios = []
    for usuario, intereses in intereses_por_usuario.items():
        valorPorUsuario = {"usuario": usuario, "valorCoincidencia": (maxPuntaje/ len(intereses))}
        valoresPorUsuarios.append(valorPorUsuario)
    return valoresPorUsuarios   
    
vCoincidenciasPorUsuario = valoresCoincidenciaPorUsuario(10)    
print(vCoincidenciasPorUsuario)

[{'usuario': 'diego3026', 'valorCoincidencia': 2.5}, {'usuario': 'diego1234', 'valorCoincidencia': 2.0}]


In [56]:
def buscarVCUsuario(nUsuario: str):
    usuario = None
    for i in range(len(vCoincidenciasPorUsuario)):
        usuNombre = vCoincidenciasPorUsuario[i]['usuario']
        if usuNombre == nUsuario:
            usuario = vCoincidenciasPorUsuario[i]
            break
    return usuario

def puntajesUsuarios():
    puntajePorUsuarios = []
    for usuario, coincidencias in coincidencias_todos_usuarios.items():
        valorCoincidenciaUsuario = buscarVCUsuario(usuario) 
        puntajesPorInmueble = []
        if valorCoincidenciaUsuario is not None:
            for coincidencia in coincidencias:
                inmuebleP = {"idimueble": coincidencia[0], "puntaje": (len(coincidencia[1]) * valorCoincidenciaUsuario["valorCoincidencia"])}
                puntajesPorInmueble.append(inmuebleP)
        puntajesInmueblesUsuario = {"usuario": usuario, "puntajePorInmueble": puntajesPorInmueble}
        puntajePorUsuarios.append(puntajesInmueblesUsuario)
    return puntajePorUsuarios

puntajesPorUsuarios = puntajesUsuarios()
print(puntajesPorUsuarios)

[{'usuario': 'diego3026', 'puntajePorInmueble': [{'idimueble': 1174, 'puntaje': 2.5}, {'idimueble': 1179, 'puntaje': 2.5}, {'idimueble': 1181, 'puntaje': 2.5}, {'idimueble': 1182, 'puntaje': 2.5}, {'idimueble': 1185, 'puntaje': 2.5}, {'idimueble': 1188, 'puntaje': 2.5}, {'idimueble': 1189, 'puntaje': 2.5}, {'idimueble': 1193, 'puntaje': 2.5}, {'idimueble': 1194, 'puntaje': 2.5}, {'idimueble': 1197, 'puntaje': 2.5}, {'idimueble': 1202, 'puntaje': 2.5}, {'idimueble': 1203, 'puntaje': 2.5}, {'idimueble': 1204, 'puntaje': 2.5}, {'idimueble': 1210, 'puntaje': 2.5}, {'idimueble': 1212, 'puntaje': 2.5}, {'idimueble': 1215, 'puntaje': 2.5}, {'idimueble': 1217, 'puntaje': 2.5}, {'idimueble': 1218, 'puntaje': 2.5}, {'idimueble': 1222, 'puntaje': 2.5}, {'idimueble': 1226, 'puntaje': 2.5}, {'idimueble': 1228, 'puntaje': 2.5}, {'idimueble': 1231, 'puntaje': 2.5}, {'idimueble': 1232, 'puntaje': 2.5}, {'idimueble': 1235, 'puntaje': 2.5}, {'idimueble': 1237, 'puntaje': 2.5}, {'idimueble': 1239, 'punta

In [57]:
data = get_datos_api()

#favorito, calificacion, clics, preferencia
pesos = [4, 2, 1, 3]

calificaciones = []
clics = []
favoritos = []

for item in data:
    if item['calificacion'] != None:
        calificaciones.append(item['calificacion'])

    if item['numeroDeClicks'] != None:
        clics.append(item['numeroDeClicks'])

    if item['favorito'] != None:
        favoritos.append(item['favorito'])

promedio_calificaciones = 1
promedio_clics = 0
promedio_favoritos = 0
max_clics = 0

if len(calificaciones) > 0:
    promedio_calificaciones = sum(calificaciones) / len(calificaciones)

if len(clics) > 0:
    max_clics = max(clics)
    promedio_clics = sum(clics) / len(clics)

if len(favoritos) > 0:
    promedio_favoritos = sum(favoritos) / len(favoritos)

for value in data:
    id_inmueble = value['inmueble'] if value['inmueble']!=None else None
    usuario = value['usuario'] if value['usuario']!=None else None
    calificacion = normalizacion(value['calificacion'],5,1) if value['calificacion']!=None else promedio_calificaciones
    favorito = int(value['favorito']) if value['favorito']!=None else promedio_favoritos

    if max_clics > 0 and value['numeroDeClicks']!=None:
        clics = normalizacion(value['numeroDeClicks'],valor_maximo=max_clics,valor_minimo=0)
    else:
        clics = promedio_clics

    preferencia = 0
    indexPuntajePorUsuarioPreferencia = next((i for i, dic in enumerate(puntajesPorUsuarios) if dic["usuario"] == usuario), -1)
    puntajesUsuario = puntajesPorUsuarios[indexPuntajePorUsuarioPreferencia]['puntajePorInmueble']
    indexPuntajePorUsuarioPreferenciaInmueble = next((i for i, dic in enumerate(puntajesUsuario) if dic["idimueble"] == id_inmueble), -1)

    if indexPuntajePorUsuarioPreferenciaInmueble != -1:
        preferencia = puntajesUsuario[indexPuntajePorUsuarioPreferenciaInmueble]['puntaje']

    if id_inmueble is not None and usuario is not None:        
        inmueble_por_usuario = InmuebleUsuarioNormalizado(inmueble=id_inmueble,
                                                        usuario=usuario,
                                                        calificacion=calificacion,
                                                        favorito=favorito,
                                                        clics=clics,
                                                        puntajePreferencia=preferencia,
                                                        pesos=pesos)
        datosLimpios[inmueble_por_usuario.usuario][inmueble_por_usuario.inmueble] = inmueble_por_usuario.get_clasificacion()

print(datosLimpios)


# Nombre del archivo donde se guardarán los datos
nombre_archivo = 'datosLimpios.json'

# Guardar el diccionario en un archivo JSON
with open(nombre_archivo, 'w') as archivo_json:
    json.dump(datosLimpios, archivo_json, indent=4)


{'diego3026': {1: 7.2, 2: 7.2, 3: 7.0, 4: 0, 5: 7.2, 7: 0, 8: 6.5, 9: 6.5, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 117: 0, 118: 0, 119: 0, 120: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 131: 0, 132: 0, 134: 0, 265: 0, 829: 0, 830: 0, 831: 0, 832: 0, 833: 0, 834:

In [58]:
import pandas as pd

ratings_df = pd.DataFrame(datosLimpios).T

In [63]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(ratings_df.fillna(0))
cosine_sim_df = pd.DataFrame(cosine_sim, index=ratings_df.index, columns=ratings_df.index)
print("Matriz de similitud del coseno:\n", cosine_sim_df)

Matriz de similitud del coseno:
            diego3026  diego1234
diego3026    1.00000    0.73267
diego1234    0.73267    1.00000


In [72]:
def predict_ratings(user, ratings_df, sim_df):
    user_ratings = ratings_df.loc[user]  # Valoraciones del usuario objetivo
    sim_scores = sim_df[user]  # Similitudes del usuario con otros usuarios
    
    # Excluir el propio usuario de la suma ponderada
    sim_scores = sim_scores.drop(user)
    ratings_df = ratings_df.drop(user)
    
    # Suma ponderada de las valoraciones de los vecinos
    weighted_sum = (ratings_df.T * sim_scores).sum(axis=1)
    sim_sum = sim_scores.sum()
    
    predicted_ratings = weighted_sum / sim_sum
    predicted_ratings = predicted_ratings[user_ratings.isna()]
    predicted_ratings = predicted_ratings[predicted_ratings > 0]

    return predicted_ratings.sort_values(ascending=False)


predicted_ratings = predict_ratings("diego3026", ratings_df, cosine_sim_df)
print("Predicciones de valoraciones:\n", predicted_ratings)

recommendations = predicted_ratings.index.tolist()
print(f"inmuebles recomendados: {recommendations}")


Predicciones de valoraciones:
 Series([], dtype: float64)
inmuebles recomendados: []
